## Import thư viện

In [1]:
import numpy as np
import pandas as pd
import time
import math
from numba import cuda
from sklearn.metrics import accuracy_score, precision_score
import warnings
warnings.filterwarnings('ignore')

## Cài đặt phiên bản XGBoost cải tiến

Áp dụng một số cải tiến giúp cải thiện hiệu suất mô hình, giảm số lần truy cập bộ nhớ, giảm thời gian thực thi nhưng vẫn đảm bảo được độ chính xác của mô hình

#### Xét hàm **`compute_split_value()`**

Tạo hàm `compute_split_value_optimize()` áp dụng các kĩ thuật cải tiến lên hàm song song hóa ban đầu

- Cải tiến 1: Trong quá trình transfer dữ liệu từ CPU/Host sang GPU/Device, Numba mặc định sẽ tự động copy dữ liệu của các tham số truyền vào hàm kernel từ Host sang Device, sau khi chạy xong hàm kernel thì sẽ copy các tham số truyền vào trước đó ngược lại về Host. Để tối ưu, ta chỉ cần copy dữ liệu cần thiết và bỏ qua dữ liệu không cần thiết khi transfer dữ liệu từ Host sang Device và Device về lại Host.
  
- Cải tiến 2: Dữ liệu của sample X nằm ở bộ nhớ GMEM, và mỗi lần đọc dữ liệu của X thì các thread đều phải chạy xuống GMEM. Vì vậy sử dụng SMEM chứa phần dữ liệu mong muốn để tính split_value. Giả sử kích thước block là 4 x 4, thì kích thước share là 5 x 4 (vì 1 $split\_value$ element cần 2 $x$ elements). Các thread sẽ lấy phần dữ liệu ứng với chỉ số `(row, col)` của chúng và ghi vào SMEM, đối với phần dữ liệu còn lại sẽ do thread liền trước đó xử lý

In [2]:
@cuda.jit
def compute_split_value_optimize(input, output):
    shared = cuda.shared.array((33, 32), dtype = np.float64)
    r = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    c = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y

    if r < output.shape[0] and c < output.shape[1]:
        shared[ty, tx] = input[r, c]

        if ty == cuda.blockDim.y - 1 or r == output.shape[0] - 1:
            shared[ty + 1, tx] = input[r + 1, c]

        cuda.syncthreads()

        if shared[ty, tx] != shared[ty + 1, tx]:
            tmp = (shared[ty, tx] + shared[ty + 1, tx]) / 2
            output[r, c] = tmp
        else:
            output[r, c] = np.nan

#### Xét hàm **`compute_gain()`**

Tạo hàm `compute_gain_optimize()` áp dụng các kĩ thuật cải tiến lên hàm song song hóa ban đầu

- Cải tiến 1: Tương tự với cải tiến 1 của hàm `compute_split_value_optimize()`, ta chỉ cần copy dữ liệu cần thiết khi transfer dữ liệu từ Host sang Device và chỉ lấy ra kết quả là dãy giá trị gain tính được từ Device về Host memory
  
- Cải tiến 2: Dữ liệu của X, residual, prob và root gain đều nằm ở bộ nhớ GMEM, mỗi lần đọc dữ liệu thì các thread đều phải chạy xuống GMEM. Do vậy sử dụng SMEM load phần dữ liệu X, residual, prob, root gain mong muốn, phục vụ cho việc so sánh giá trị và tính gain. Với root gain, ta sẽ để thread có chỉ số `(tx = 0, ty = 0)` load dữ liệu vào SMEM. Với các dữ liệu còn lại, ta sẽ chia phần dữ liệu mà block cần thành nhiều phần nhỏ. Ở lần xử lý đầu tiên, block sẽ load một phần nhỏ của dữ liệu từ GMEM vào SMEM, rồi mỗi thread trong block đọc dữ liệu ở SMEM để tính một phần kết quả; ở lần xử lý kế, block sẽ load phần nhỏ tiếp theo (ghi đè lên dữ liệu cũ) và tính tiếp từ kết quả đang tính trước đó, tiếp tục cho đến hết.

In [3]:
@cuda.jit
def compute_gain_optimize(split, output, X, residuals, p, lambda_, min_child_weight, min_samples, root_gain):
    c, r = cuda.grid(2)
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    shared_X = cuda.shared.array((32, 32), dtype = np.float64)
    shared_residual = cuda.shared.array((32), dtype = np.float64)
    shared_prob = cuda.shared.array((32), dtype = np.float64)
    shared_root_gain = cuda.shared.array((1), dtype = np.float64)
    left_nu, right_nu, left_de, right_de, left_samples, right_samples = 0, 0, 0, 0, 0, 0

    if r < split.shape[0] and c < split.shape[1]:
        split_val = split[r, c]
    else:
        split_val = np.nan
    if tx == 0 and ty == 0:
        shared_root_gain[0] = root_gain

    for phase in range(math.ceil(X.shape[0] / 32)):
        if c < X.shape[1] and phase * 32 + ty < X.shape[0]:
            shared_X[ty, tx] = X[phase * 32 + ty, c]
            if tx == 0:
                shared_residual[ty] = residuals[phase * 32 + ty]
                shared_prob[ty] = p[phase * 32 + ty]
        cuda.syncthreads()

        if split_val != np.nan:
            for i in range(shared_X.shape[0]):
                if phase * 32 + i < X.shape[0]:
                    if shared_X[i, tx] <= split_val:
                        left_nu += shared_residual[i]
                        left_de += shared_prob[i] * (1 - shared_prob[i])
                        left_samples += 1
                    else:
                        right_nu += shared_residual[i]
                        right_de += shared_prob[i] * (1 - shared_prob[i])
                        right_samples += 1
            cuda.syncthreads()

    if r < split.shape[0] and c < split.shape[1]:
        if (left_samples < min_samples or right_samples < min_samples
            or left_de < min_child_weight or right_de < min_child_weight ):
            output[r, c] = np.nan
        else:
            left_sim = (left_nu ** 2) / (left_de + lambda_)
            right_sim = (right_nu ** 2) / (right_de + lambda_)
            gain = left_sim + right_sim - shared_root_gain[0]
            output[r, c] = gain

In [4]:
class Tree:
    def __init__(self, max_depth = 3, min_samples = 1, min_child_weight = 1, lambda_ = 0, gamma = 0):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.min_child_weight = min_child_weight
        self.lambda_ = lambda_
        self.gamma = gamma
        self.tree = {}
        self.fbs_time = 0

    def similarity(self, residual, probs):
        nu = np.sum(residual) ** 2
        de = np.sum(probs * (1 - probs)) + self.lambda_
        return nu / de

    def compute_output(self, residual, probs):
        nu = np.sum(residual)
        de = np.sum(probs * (1 - probs)) + self.lambda_
        return nu / de

    def split_data(self, X, feature_idx, split_value):
        left_idx = X[:, feature_idx] <= split_value
        right_idx = X[:, feature_idx] > split_value
        return left_idx, right_idx

    def find_best_split(self, X, residuals, probs):
        best_gain = -np.inf
        best_split_feature_idx = None
        best_split_value = None

        split_array = np.empty((X.shape[0] - 1, X.shape[1]))
        #allocate and transfer data from Host to Device memory
        d_split_array = cuda.device_array(split_array.shape)
        # d_X = cuda.to_device(np.sort(X, axis = 0).astype(np.float64))
        d_X = cuda.to_device(X)

        block_size = (32, 32)
        grid_split = (math.ceil(split_array.shape[1] / block_size[0]),
                      math.ceil(split_array.shape[0] / block_size[1]))
        # compute_split_value_optimize[grid_split, block_size](np.sort(X, axis=0).astype(np.float64), split_array)
        compute_split_value_optimize[grid_split, block_size](np.sort(d_X, axis=0).astype(np.float64), d_split_array)
        #transfer result back to Host
        split_array = d_split_array.copy_to_host()

        root_gain = self.similarity(residuals, probs)
        gain_array = np.empty((split_array.shape[0], split_array.shape[1]))
        #allocate and transfer data from host to device memory
        d_split_array = cuda.to_device(split_array)
        d_gain_array = cuda.device_array(gain_array.shape)
        d_residuals = cuda.to_device(residuals)
        d_probs = cuda.to_device(probs)
        grid_gain = (math.ceil(gain_array.shape[1] / block_size[0]),
                     math.ceil(gain_array.shape[0] / block_size[1]))

        # compute_gain_optimize[grid_gain, block_size](split_array, gain_array, X, residuals,
        #                                     probs, self.lambda_, self.min_child_weight, self.min_samples, root_gain)

        compute_gain_optimize[grid_gain, block_size](d_split_array, d_gain_array, d_X, d_residuals,
                                            d_probs, self.lambda_, self.min_child_weight, self.min_samples, root_gain)
        #transfer result back to Host
        gain_array = d_gain_array.copy_to_host()

        gain_array[np.isnan(gain_array)] = -np.inf
        if np.sum(gain_array == -np.inf) != (gain_array.shape[0] * gain_array.shape[1]):
            tmp_gain = np.transpose(gain_array)
            max_idx = np.unravel_index(tmp_gain.argmax(), tmp_gain.shape)
            final_idx = (max_idx[1], max_idx[0])
            best_split_feature_idx = final_idx[1]
            best_gain = gain_array[final_idx]
            best_split_value = split_array[final_idx]

        if(best_gain - self.gamma < 0):
            best_split_feature_idx = None
            best_split_value = None

        return best_split_feature_idx, best_split_value

    def build_tree(self, X, residual, probs, depth):
        if depth >= self.max_depth or len(X) <= self.min_samples:
            return self.compute_output(residual, probs)

        start = time.time()
        split_feature_idx, split_value = self.find_best_split(X, residual, probs)
        end = time.time()
        self.fbs_time += (end - start)

        if split_feature_idx is None:
            return self.compute_output(residual, probs)

        left_idx, right_idx = self.split_data(X, split_feature_idx, split_value)
        left = self.build_tree(X[left_idx], residual[left_idx], probs[left_idx], depth + 1)
        right = self.build_tree(X[right_idx], residual[right_idx], probs[right_idx], depth + 1)

        self.tree = {
            'split_feature_idx': split_feature_idx,
            'split_value': split_value,
            'left_child': left,
            'right_child': right
        }
        return self.tree

    def get_output(self, x, tree):
        if isinstance(tree, dict):
            split_feature_idx = tree['split_feature_idx']
            split_value = tree['split_value']
            if x[split_feature_idx] <= split_value:
                return self.get_output(x, tree['left_child'])
            else:
                return self.get_output(x, tree['right_child'])
        else:
            return tree

    def fit(self, X, residual, probs):
        depth = 0
        self.tree = self.build_tree(X, residual, probs, depth)

    def predict(self, X):
        return np.array([self.get_output(x, self.tree) for x in X])

#### Xét các hàm **`residual_optimize()`**, **`compute_logodds_optimize()`**, **`compute_prob_optimize()`**

Các hàm `residual_optimize()` , `compute_logodds_optimize()`, `compute_prob_optimize()` sẽ cho mỗi thread block xử lý $2 * blockDim.x$ phần tử. Tất cả các thread trong mỗi block sẽ xử lý $blockDim.x$ phần tử đầu mảng, mỗi thread xử lý một phần tử, sau đó tất cả các thread sẽ chuyển sang $blockDim.x$ phần tử sau của mảng, mỗi thread xử lý một phần tử. Tương tự với các hàm tối ưu trước, chỉ copy dữ liệu cần thiết khi transfer dữ liệu từ Host sang Device và copy mảng kết quả về Host memory

In [5]:
@cuda.jit
def residual_kernel(y_true, y_pred, residual):
    i = cuda.blockIdx.x * cuda.blockDim.x * 2 + cuda.threadIdx.x
    if i < residual.shape[0]:
        residual[i] = y_true[i] - y_pred[i]
    if i + cuda.blockDim.x < residual.shape[0]:
        residual[i + cuda.blockDim.x] = y_true[i + cuda.blockDim.x] - y_pred[i + cuda.blockDim.x]

In [6]:
@cuda.jit
def compute_logodds_kernel(p, log_odds):
    i = cuda.blockIdx.x * cuda.blockDim.x * 2 + cuda.threadIdx.x
    if i < log_odds.shape[0]:
        log_odds[i] = math.log(p[i] / (1 - p[i]))
    if i + cuda.blockDim.x < log_odds.shape[0]:
        log_odds[i + cuda.blockDim.x] = math.log(p[i + cuda.blockDim.x] / (1 - p[i + cuda.blockDim.x]))

In [7]:
@cuda.jit
def compute_prob_kernel(logodds_p, p):
    i = cuda.blockIdx.x * cuda.blockDim.x * 2 + cuda.threadIdx.x
    if i < p.shape[0]:
        p[i] = math.exp(logodds_p[i]) / (1 + math.exp(logodds_p[i]))
    if i + cuda.blockDim.x < p.shape[0]:
        p[i + cuda.blockDim.x] = math.exp(logodds_p[i + cuda.blockDim.x]) / (1 + math.exp(logodds_p[i + cuda.blockDim.x]))

In [8]:
class XGBoost:
    def __init__(self, n_estimators, lr, lambda_ = 0, gamma = 0, min_child_weight = 1, max_depth = 3):
        self.n_estimators = n_estimators
        self.lr = lr
        self.initial_pred = 0.5
        self.lambda_ = lambda_
        self.min_child_weight = min_child_weight
        self.max_depth = max_depth
        self.gamma = gamma
        self.models = []
        self.fbs_time = 0
        self.logodds_time = 0
        self.residual_time = 0
        self.predict_time = 0

    def fit(self, X, y):
        p = np.full(len(y), self.initial_pred)
        block_size = 32
        grid_size = math.ceil(len(y) / 2 * block_size)

        for _ in range(self.n_estimators):
            probs = np.copy(p)

            residual = np.empty(len(y))
            d_y = cuda.to_device(y)
            d_p = cuda.to_device(p)
            d_residual = cuda.device_array(residual.shape)
            start = time.time()
            # residual_kernel[grid_size, block_size](y, p, residual)
            residual_kernel[grid_size, block_size](d_y, d_p, d_residual)
            residual = d_residual.copy_to_host()

            end = time.time()
            self.residual_time += (end - start)

            model = Tree(lambda_ = self.lambda_, gamma = self.gamma, max_depth = self.max_depth, min_child_weight = self.min_child_weight)
            model.fit(X, residual, probs)
            self.fbs_time += model.fbs_time

            log_odds = np.empty(len(y))
            d_log_odds = cuda.device_array(log_odds.shape)
            start = time.time()
            # compute_logodds_kernel[grid_size, block_size](p, log_odds)
            compute_logodds_kernel[grid_size, block_size](d_p, d_log_odds)
            log_odds = d_log_odds.copy_to_host()
            end = time.time()
            self.logodds_time += (end - start)

            p = np.empty(len(y))
            d_p = cuda.device_array(p.shape)
            start = time.time()
            logodds_p = log_odds + self.lr * model.predict(X)
            d_logodds_p = cuda.to_device(logodds_p)
            # compute_prob_kernel[grid_size, block_size](logodds_p, p)
            compute_prob_kernel[grid_size, block_size](d_logodds_p, d_p)
            p = d_p.copy_to_host()
            end = time.time()
            self.predict_time += (end - start)

            self.models.append(model)

    def predict_proba(self, X):
        pred = np.full(len(X), self.initial_pred)
        block_size = 32
        grid_size = math.ceil(len(X) / 2 * block_size)
        for model in self.models:
            log_odds = np.empty(len(X))
            d_log_odds = cuda.device_array(log_odds.shape)
            d_pred = cuda.to_device(pred)
            # compute_logodds_kernel[grid_size, block_size](pred, log_odds)
            compute_logodds_kernel[grid_size, block_size](d_pred, d_log_odds)
            log_odds = d_log_odds.copy_to_host()
            logodds_p = log_odds + self.lr * model.predict(X)
            pred = np.empty(len(X))
            d_logodds_p = cuda.to_device(logodds_p)
            d_pred = cuda.device_array(pred.shape)
            # compute_prob_kernel[grid_size, block_size](logodds_p, pred)
            compute_prob_kernel[grid_size, block_size](d_logodds_p, d_pred)
            pred = d_pred.copy_to_host()
        return pred

## Xây dựng mô hình đa lớp

In [9]:
class MultiClassifier:
    def __init__(self, n_estimators = 3, lr = 0.3):
        self.models = []
        self.n_estimators = n_estimators
        self.lr = lr
        self.training_time = 0

    def fit(self, X, y):
        start_time = time.time()
        for label in np.unique(y):
            binary_labels = (y == label).astype(int)
            model = XGBoost(self.n_estimators, self.lr)
            model.fit(X, binary_labels)
            self.models.append(model)
        end_time = time.time()
        self.training_time += (end_time - start_time)

    def predict(self, X):
        preds = []
        for model in self.models:
            preds.append(model.predict_proba(X))
        return np.argmax(preds, axis = 0)

## Chạy mô hình

In [10]:
train = np.load('train_data_3labels.npz', allow_pickle = True)
X_train = train['data']
y_train = train['label']

test = np.load('test_data_3labels.npz', allow_pickle = True)
X_test = test['data']
y_test = test['label']

#### Binary classification

In [12]:
binary_classifier = XGBoost(n_estimators = 3, lr = 0.3)
start_time = time.time()
binary_classifier.fit(X_train, (y_train == 0).astype(int))
end_time = time.time()

y_prob_pred = binary_classifier.predict_proba(X_test)
binary_labels_pred = (y_prob_pred > 0.5).astype(int)
binary_labels_test = (y_test == 0).astype(int)

accuracy_binary = accuracy_score(binary_labels_test, binary_labels_pred)
time_binary = end_time - start_time

print('Accuracy:', accuracy_binary)
print(f'Total time: {time_binary} seconds')

Accuracy: 0.9466666666666667
Total time: 2.5445396900177 seconds


#### Multi classification

In [14]:
multi_classifier = MultiClassifier()
multi_classifier.fit(X_train, y_train)

y_pred = multi_classifier.predict(X_test)
accuracy_3labels = accuracy_score(y_test, y_pred)
time_3labels = multi_classifier.training_time

print('Accuracy:', accuracy_3labels)
print(f'Total time: {time_3labels} seconds')

Accuracy: 0.9566666666666667
Total time: 7.652122974395752 seconds


#### Multi classification trên tập dữ liệu hoàn chỉnh (10 lớp)

In [15]:
train = np.load('train_data.npz', allow_pickle = True)
X_train = train['data']
y_train = train['label']

test = np.load('test_data.npz', allow_pickle = True)
X_test = test['data']
y_test = test['label']

In [16]:
multi_classifier = MultiClassifier(n_estimators = 35, lr = 0.3)
multi_classifier.fit(X_train, y_train)

y_pred = multi_classifier.predict(X_test)
accuracy_10labels = accuracy_score(y_test, y_pred)
time_10label = multi_classifier.training_time

print('Accuracy:', accuracy_10labels)
print(f'Total time: {time_10label} seconds')

Accuracy: 0.891
Total time: 2589.382884502411 seconds


**Kết luận:** Phiên bản optimize đã cải thiện tốt thời gian huấn luyện nhưng vẫn đảm bảo được độ chính xác trong mọi trường hợp

## Lưu kết quả

In [17]:
result_df = pd.read_csv('result.csv')

In [18]:
result_df['Optimize_training_time'] = [time_binary, time_3labels, time_10label]
result_df['Optimize_accuracy'] = [accuracy_binary, accuracy_3labels, accuracy_10labels]

In [20]:
result_df.to_csv('result.csv', index = False)

## Under sampling

In [21]:
class MultiClassifier_2:
    def __init__(self, n_estimators = 3, lr = 0.3):
        self.models = []
        self.n_estimators = n_estimators
        self.lr = lr
        self.training_time = 0.0

    def under_sampling(self, X, y):
        size = int((np.count_nonzero(y)) * 1.5)
        random_idx = np.random.choice(len(np.where(y == 0)[0]), size = size)
        idx = np.concatenate([np.where(y == 1)[0], random_idx])
        return X[idx], y[idx]

    def fit(self, X, y):
        start_time = time.time()
        for label in np.unique(y):
            binary_labels = (y == label).astype(int)
            X_balance, y_balance = self.under_sampling(X, binary_labels)
            model = XGBoost(self.n_estimators, self.lr)
            model.fit(X_balance, y_balance)
            self.models.append(model)
        end_time = time.time()
        self.training_time += (end_time - start_time)

    def predict(self, X):
        preds = []
        for model in self.models:
            preds.append(model.predict_proba(X))
        return np.argmax(preds, axis = 0)

In [23]:
multi_classifier = MultiClassifier_2(n_estimators = 100, lr = 0.3)
multi_classifier.fit(X_train, y_train)

y_pred = multi_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = None)
training_time = multi_classifier.training_time

print('Accuracy:', accuracy)
print('Precision:', precision)
print(f'Total time: {training_time} seconds')

Accuracy: 0.924
Precision: [0.95145631 0.87272727 0.82142857 0.95744681 0.96551724 0.87037037
 0.91666667 0.94949495 0.98989899 0.97826087]
Total time: 601.2597625255585 seconds


Sử dụng under sampling trên phiên bản optimize giúp cho mô hình chạy với thời gian nhanh hơn, accuracy và precision cho thấy kết quả dự đoán của mô hình khá tốt.